# Template
> Define the skeleton of an algorithm in an operation, deferring some steps to subclasses. Template method lets subclasses redefine certain steps of an algorithm without changing the algorithm's structure.

Keep the basic code same, outsource the varying parts.

## Problem
Consider an ML problem setting. Lets say I have a metrics aggregator that finds the arithmetic mean of the accuracy over the last 5 mini batches.

In [7]:
class AccuracyAgg:
    def sample(self, metrics):
        means = []
        for step in range(0, len(metrics), 5): 
            grp = metrics[step:step+5]
            mean = sum(grp) / len(grp)
            means.append(mean)
        return means

In [8]:
acc = AccuracyAgg()
acc.sample([10, 11, 4, 8, 19, 9, 7, 12, 4, 6])

[10.4, 7.6]

Now, lets say that I need to find the harmonic mean of the precision and recall every 3 mini batches. I could of course write another aggregator class. But that would not scale well with different types of metrics, aggregation logic, and mini batches.

## Solution

### OO Solution
The basic template of the logic is common - I need to group the metrics together over some number of mini batches, and then apply some aggregation function to each member of this group. I can keep this template in a common `MetricsAggregator` class and have the actual grouping and aggregating functionality live in different subclasses. This way I can keep reusing the common code in a lot of different settings.

In [10]:
from abc import ABC, abstractmethod

class MetricsAgg(ABC):
    @abstractmethod
    def group(self, metrics):
        pass
    
    @abstractmethod
    def aggregate(self, group):
        pass
    
    def sample(self, metrics):
        aggs = []
        for grp in self.group(metrics):
            agg = self.aggregate(grp)
            aggs.append(agg)
        return aggs
    
    
class AccuracyAgg(MetricsAgg):
    def group(self, metrics):
        for step in range(0, len(metrics), 5):
            yield metrics[step:step+5]
            
    def aggregate(self, group):
        return sum(group) / len(group)

In [11]:
acc = AccuracyAgg()
acc.sample([10, 11, 4, 8, 19, 9, 7, 12, 4, 6])

[10.4, 7.6]

### Pythonic Solution
Instead of creating abstract methods that are implemented in sub-classes, I can simply pass in two functions to the `sample` method for grouping and aggregating the metrics.

In [12]:
class MetricsAgg:
    def __init__(self, grouper, aggregator):
        self.grouper = grouper
        self.aggregator = aggregator
        
    def sample(self, metrics):
        aggs = []
        for grp in self.grouper(metrics):
            agg = self.aggregator(grp)
            aggs.append(agg)
        return aggs

In [13]:
def take5(metrics):
    for step in range(0, len(metrics), 5):
        yield metrics[step:step+5]

def mean(group):
    return sum(group) / len(group)

acc = MetricsAgg(take5, mean)
acc.sample([10, 11, 4, 8, 19, 9, 7, 12, 4, 6])

[10.4, 7.6]